# 基于MindSpore的GPT2文本摘要

## 环境配置

1. 安装tokenizers，版本为0.15.0；

2. 需要手动安装mindspore2.2.13，npu版本，python环境为3.9，安装指南详见：[MindSpore安装](https://www.mindspore.cn/install)

3. 安装MindNLP及相关依赖，MindNLP官方仓详见：[MindNLP](https://github.com/mindspore-lab/mindnlp)

4. 安装pytest，版本要大于7.2；

In [1]:
!pip install mindspore-2.2.13-cp39-cp39-linux_aarch64.whl -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Processing ./mindspore-2.2.13-cp39-cp39-linux_aarch64.whl
  Attempting uninstall: mindspore
    Found existing installation: mindspore 2.2.0
    Uninstalling mindspore-2.2.0:
      Successfully uninstalled mindspore-2.2.0
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
!pip install mindnlp-0.2.0.20240229-py3-none-any.whl

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
Processing ./mindnlp-0.2.0.20240229-py3-none-any.whl
     |████████████████████████████████| 510 kB 51.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.5 MB 45.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 62.9 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 62.4 MB/s eta 0:00:01
     |████████████████████████████████| 220 kB 60.6 MB/s eta 0:00:01
     |████████████████████████████████| 1.2 MB 58.7 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 70.3 MB/s eta 0:00:01
     |████████████████████████████████| 35.7 MB 59.0 MB/s eta 0:00:01
     |████████████████████████████████| 300 kB 64.8 MB/s eta 0:00:01
     |████████████████████████████████| 125 kB 43.1 MB/s eta 0:00:01
     |████████████████████████████████| 239 kB 65.4 MB/s eta 0:00:01
     |████████████████████████████████| 116 kB 64.4 MB/s eta 0:00:01
  Attempting uninstall: dill
    Foun

In [2]:
!pip install --upgrade pip

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
     |████████████████████████████████| 2.1 MB 58.7 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.0.1
    Uninstalling pip-21.0.1:
      Successfully uninstalled pip-21.0.1


In [3]:
!pip install pytest==7.2.2

Looking in indexes: http://pip.modelarts.private.com:8888/repository/pypi/simple
     |████████████████████████████████| 317 kB 60.1 MB/s eta 0:00:01
  Attempting uninstall: pytest
    Found existing installation: pytest 7.1.2
    Uninstalling pytest-7.1.2:
      Successfully uninstalled pytest-7.1.2
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python3.9 -m pip install --upgrade pip' command.


In [4]:
!pip install tokenizers==0.15.0 -i https://pypi.tuna.tsinghua.edu.cn/simple
#!wget https://repo.mindspore.cn/mindspore-lab/mindnlp/daily/202402/20240229/master_20240229160016_c5444092d8cfe47d73e292f25d9a9a56fb04828a_newest/any/mindnlp-0.2.0.20240229-py3-none-any.whl
#!pip install mindnlp-0.2.0.20240229-py3-none-any.whl
#!pip install pytest==7.2.*

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 3.8 MB 7.2 MB/s eta 0:00:01
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
You should consider upgrading via the '/home/ma-user/anaconda3/envs/MindSpore/bin/python3.9 -m pip install --upgrade pip' command.


### 数据集加载与处理

1. 数据集加载

    本次实验使用的是nlpcc2017摘要数据，内容为新闻正文及其摘要，总计50000个样本，为加快训练速度这里只选取100条数据。

In [5]:
import os

from mindnlp.utils import http_get

path = "./train_with_summ.txt"
if os.path.exists(path):  
    print("文件存在!")  
else:  
    url = 'https://download.mindspore.cn/toolkits/mindnlp/dataset/text_generation/nlpcc2017/train_with_summ.txt'
    path = http_get(url, './')

[WARNING] ME(1308:281472967340208,MainProcess):2024-04-11-17:35:12.932.854 [mindspore/run_check/_check_version.py:357] MindSpore version 2.2.13 and Ascend AI software package (Ascend Data Center Solution)version 7.0 does not match, the version of software package expect one of ['7.1']. Please refer to the match info on: https://www.mindspore.cn/install
/usr/local/Ascend/ascend-toolkit/7.0.RC1/python/site-packages/tbe/tvm/contrib/ccec.py:766: DeprecationWarning: invalid escape sequence \L
  if not dirpath.find("AppData\Local\Temp"):
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/classifier/transdata/transdata_classifier.py:222: DeprecationWarning: invalid escape sequence \B
  """
/usr/local/Ascend/ascend-toolkit/latest/python/site-packages/tbe/dsl/unify_schedule/vector/transdata/common/graph/transdata_graph_info.py:140: DeprecationWarning: invalid escape sequence \c
  """
/home/ma-user/anaconda3/envs/MindSpore/lib/python3.9/site-packages/numpy/core/getlimits.py:499

文件存在!


In [6]:
from mindspore.dataset import TextFileDataset

# dataset = TextFileDataset(str(path), shuffle=False)
dataset = TextFileDataset(str(path), num_samples=100, shuffle=False)
dataset.get_dataset_size()

100

In [7]:
train_dataset, test_dataset = dataset.split([0.9, 0.1], randomize=False)

2. 数据预处理

    原始数据格式：
    ```text
    article: [CLS] article_context [SEP]
    summary: [CLS] summary_context [SEP]
    ```
    预处理后的数据格式：

    ```text
    [CLS] article_context [SEP] summary_context [SEP]
    ```

In [8]:
import json
import numpy as np

def process_dataset(dataset, tokenizer, batch_size=6, max_seq_len=1024, shuffle=False):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def merge_and_pad(article, summary):
        tokenized = tokenizer(text=article, text_pair=summary,
                              padding='max_length', truncation='only_first', max_length=max_seq_len)
        return tokenized['input_ids'], tokenized['input_ids']
    
    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(merge_and_pad, ['article', 'summary'], ['input_ids', 'labels'])

    dataset = dataset.batch(batch_size)
    if shuffle:
        dataset = dataset.shuffle(batch_size)

    return dataset

In [9]:
from mindnlp.transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

因GPT2无中文的tokenizer，我们使用BertTokenizer替代。

In [10]:
train_dataset = process_dataset(train_dataset, tokenizer, batch_size=4)

In [11]:
next(train_dataset.create_tuple_iterator())

[Tensor(shape=[4, 1024], dtype=Int64, value=
 [[ 101, 1724, 3862 ...    0,    0,    0],
  [ 101,  704, 3173 ...    0,    0,    0],
  [ 101, 1079, 2159 ... 1745, 8021,  102],
  [ 101, 1355, 2357 ...    0,    0,    0]]),
 Tensor(shape=[4, 1024], dtype=Int64, value=
 [[ 101, 1724, 3862 ...    0,    0,    0],
  [ 101,  704, 3173 ...    0,    0,    0],
  [ 101, 1079, 2159 ... 1745, 8021,  102],
  [ 101, 1355, 2357 ...    0,    0,    0]])]

In [12]:
len(tokenizer)

21128

### 模型构建

1. 构建GPT2ForSummarization模型，注意***shift right***的操作。

In [13]:
from mindspore import ops
from mindnlp.transformers import GPT2LMHeadModel

class GPT2ForSummarization(GPT2LMHeadModel):
    def construct(
        self,
        input_ids = None,
        attention_mask = None,
        labels = None,
    ):
        outputs = super().construct(input_ids=input_ids, attention_mask=attention_mask)
        shift_logits = outputs.logits[..., :-1, :]
        shift_labels = labels[..., 1:]
        # Flatten the tokens
        loss = ops.cross_entropy(shift_logits.view(-1, shift_logits.shape[-1]), shift_labels.view(-1), ignore_index=tokenizer.pad_token_id)
        return loss

2. 设置动态学习率

In [14]:
from mindspore import ops
from mindspore.nn.learning_rate_schedule import LearningRateSchedule

class LinearWithWarmUp(LearningRateSchedule):
    """
    Warmup-decay learning rate.
    """
    def __init__(self, learning_rate, num_warmup_steps, num_training_steps):
        super().__init__()
        self.learning_rate = learning_rate
        self.num_warmup_steps = num_warmup_steps
        self.num_training_steps = num_training_steps

    def construct(self, global_step):
        if global_step < self.num_warmup_steps:
            return global_step / float(max(1, self.num_warmup_steps)) * self.learning_rate
        return ops.maximum(
            0.0, (self.num_training_steps - global_step) / (max(1, self.num_training_steps - self.num_warmup_steps))
        ) * self.learning_rate

### 模型训练

设置模型训练相关参数

In [15]:
num_epochs = 1
warmup_steps = 20
learning_rate = 1.5e-4

num_training_steps = num_epochs * train_dataset.get_dataset_size()

In [16]:
from mindspore import nn
from mindnlp.transformers import GPT2Config, GPT2LMHeadModel

config = GPT2Config(vocab_size=len(tokenizer))
model = GPT2ForSummarization(config)

lr_scheduler = LinearWithWarmUp(learning_rate=learning_rate, num_warmup_steps=warmup_steps, num_training_steps=num_training_steps)
optimizer = nn.AdamWeightDecay(model.trainable_params(), learning_rate=lr_scheduler)

In [17]:
# 记录模型参数数量
print('number of model parameters: {}'.format(model.num_parameters()))

number of model parameters: 102068736


In [18]:
from mindnlp.engine import Trainer
from mindnlp.engine.callbacks import CheckpointCallback

ckpoint_cb = CheckpointCallback(save_path='checkpoint', ckpt_name='gpt2_summarization',
                                epochs=1, keep_checkpoint_max=2)

trainer = Trainer(network=model, train_dataset=train_dataset,
                  epochs=1, optimizer=optimizer, callbacks=ckpoint_cb)
trainer.set_amp(level='O1')

Trainer will use 'StaticLossScaler' with `scale_value=2 ** 10` when `loss_scaler` is None.


In [ ]:
trainer.run(tgt_columns="labels")

The train will start from the checkpoint saved in 'checkpoint'.


Epoch 0:   0%|          | 0/23 [00:00<?, ?it/s]

### 模型推理

数据处理，将向量数据变为中文数据

In [18]:
def process_test_dataset(dataset, tokenizer, batch_size=1, max_seq_len=1024, max_summary_len=100):
    def read_map(text):
        data = json.loads(text.tobytes())
        return np.array(data['article']), np.array(data['summarization'])

    def pad(article):
        tokenized = tokenizer(text=article, truncation=True, max_length=max_seq_len-max_summary_len)
        return tokenized['input_ids']

    dataset = dataset.map(read_map, 'text', ['article', 'summary'])
    dataset = dataset.map(pad, 'article', ['input_ids'])
    
    dataset = dataset.batch(batch_size)

    return dataset

In [19]:
batched_test_dataset = process_test_dataset(test_dataset, tokenizer, batch_size=1)

In [20]:
print(next(batched_test_dataset.create_tuple_iterator(output_numpy=True)))

[array([[ 101, 3173, 1290, 5381, 1266,  776,  128, 3299, 8176, 3189, 4510,
         133,  100,  135, 2945, 3173, 1290, 4852,  100, 3173, 1290, 1744,
        7354,  100, 2145, 2787, 4999, 2845, 6887, 8024, 2945, 7350, 5468,
        6979,  517, 3862, 3968, 3173, 7319, 2845,  518, 8162, 3189, 2845,
        6887, 8024, 6832, 2876, 6356, 3175, 3189, 1184, 4788, 5815,  749,
         671,  702, 4507,  127, 1399,  704, 1744, 5093, 4511, 2094, 5299,
        2768, 4638, 1057, 2147, 4668, 4961, 1730,  832, 8024,  800,  812,
        1762,  791, 2399,  127, 3299, 5635,  128, 3299, 7313,  754, 6832,
        2876,  510, 7350, 2357, 2799, 3683,  510, 3763, 6830, 5023, 6979,
        7270, 1744, 4638, 1166, 1863, 1079, 4556, 4312,  868, 3428, 5276,
        8203, 6629, 8024, 4668, 4961, 4403, 2140,  510, 1399, 6134,  510,
        4385, 7032, 5023, 6586, 7028, 6568, 4289, 2600,  817,  966, 6809,
        3144, 4636,  674, 6832, 2861, 1990, 8020,  122, 6832, 2861, 1990,
        5276, 1394,  122, 8026,  128,

In [22]:
model = GPT2LMHeadModel.from_pretrained('./checkpoint/gpt2_summarization_epoch_0.ckpt', config=config)

Generation config file not found, using a generation config created from the model config.


In [ ]:
model.set_train(False)
model.config.eos_token_id = model.config.sep_token_id
i = 0
for (input_ids, raw_summary) in batched_test_dataset.create_tuple_iterator():
    output_ids = model.generate(input_ids, max_new_tokens=50, num_beams=5, no_repeat_ngram_size=2)
    output_text = tokenizer.decode(output_ids[0].tolist())
    print(output_text)
    i += 1
    if i == 1:
        break